In [0]:
#Importing first essential libraries

import cv2
import numpy as np
from tqdm import tqdm
from random import shuffle
import pandas as pd
import matplotlib.pyplot as plt
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


In [0]:
#Mounting Google Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading data

In [0]:
#load our test and train data

batch_size = 64

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of ''/content/drive/My Drive/14W/Train'', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/14W/Train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for test data
test_generator = test_datagen.flow_from_directory(
        '/content/drive/My Drive/14W/Test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 254 images belonging to 2 classes.
Found 64 images belonging to 2 classes.


# Model Building

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

# Fit Data

In [0]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=test_generator,
        validation_steps=800 // batch_size)
model.save_weights('first_try.h5')  # always save your weights after training or during training

Epoch 1/50
31/31 [==============================] - 55s 2s/step - loss: 0.6812 - acc: 0.5984 - val_loss: 0.7003 - val_acc: 0.5156
Epoch 2/50
31/31 [==============================] - 27s 857ms/step - loss: 0.6096 - acc: 0.6663 - val_loss: 0.8933 - val_acc: 0.4688
Epoch 3/50
31/31 [==============================] - 26s 841ms/step - loss: 0.5345 - acc: 0.7252 - val_loss: 1.1216 - val_acc: 0.4531
Epoch 4/50
31/31 [==============================] - 26s 836ms/step - loss: 0.4290 - acc: 0.7990 - val_loss: 1.3498 - val_acc: 0.3594
Epoch 5/50
31/31 [==============================] - 25s 791ms/step - loss: 0.3194 - acc: 0.8601 - val_loss: 2.3648 - val_acc: 0.4531
Epoch 6/50
31/31 [==============================] - 27s 856ms/step - loss: 0.2597 - acc: 0.9005 - val_loss: 2.2981 - val_acc: 0.3125
Epoch 7/50
31/31 [==============================] - 26s 850ms/step - loss: 0.2051 - acc: 0.9198 - val_loss: 2.6344 - val_acc: 0.3125
Epoch 8/50
31/31 [==============================] - 26s 827ms/step - los

# Evaluation

In [0]:
true_classes = test_generator.classes
print(len(true_classes))


predictions = model.predict_generator(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
print(predicted_classes)

64
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
